In [1]:
import pandas as pd


# Procesamiento del Dataset

In [2]:
#importamos el dataset
dataset = pd.read_excel("./src/lenses.ods")

#seleccionamos la condición de ejemplo positivo
target = dataset['target']
corpus = dataset.drop(['target'], axis=1).drop(['#'], axis=1)

print(dataset)
print(corpus)
print(target)

     #  Age  Spectacle_prescription  Astigmatic  Tear_production_rate  target
0    1    1                       1           1                     1       3
1    2    1                       1           1                     2       2
2    3    1                       1           2                     1       3
3    4    1                       1           2                     2       1
4    5    1                       2           1                     1       3
5    6    1                       2           1                     2       2
6    7    1                       2           2                     1       3
7    8    1                       2           2                     2       1
8    9    2                       1           1                     1       3
9   10    2                       1           1                     2       2
10  11    2                       1           2                     1       3
11  12    2                       1           2                 

In [3]:
#probemos que podemos usar la librería que creamos

from procesamiento_datasets import valores_observados, repeticiones_de_valor, extraer_muestra

In [4]:
#generamos el conjunto de entrenamiento y el conjunto de prueba

total_clases_original = []

clases = valores_observados(target.values)
proporcion = []

for clase in clases:
    total = repeticiones_de_valor(clase, target.values)
    total_clases_original.append([clase, total])

    proporcion.append([clase, total/len(target.values)])

print(total_clases_original)
print(clases)
print(proporcion)


[[3, 15], [2, 5], [1, 4]]
[3, 2, 1]
[[3, 0.625], [2, 0.20833333333333334], [1, 0.16666666666666666]]


In [5]:
#obtenemos la muestra basandonos en los datos anteriores.

x_muestra, y_muestra, x_validacion, y_validacion = extraer_muestra(corpus, target, proporcion, len(target.values), (2/3))

print(x_muestra)
print(y_muestra)

print(x_validacion)
print(y_validacion)

# print(len(x_muestra), len(y_muestra))

# print(len(x_muestra)/(len(x_muestra) + len(y_muestra)))

clases_muestra = valores_observados(y_muestra["target"].values)
total_clases_muestra = []

for clase in clases_muestra:
    total = repeticiones_de_valor(clase, y_muestra["target"].values)
    total_clases_muestra.append([clase, total])

print(total_clases_muestra)

16
[[3, 10], [2, 3], [1, 2]]
[3, 10]
15 15
[2, 3]
10 10
[1, 2]
11 11
    Age  Spectacle_prescription  Astigmatic  Tear_production_rate
0     3                       2           2                     1
1     3                       2           2                     2
2     2                       1           1                     1
3     3                       2           1                     1
4     1                       1           2                     1
5     2                       2           2                     1
6     3                       1           1                     1
7     1                       2           2                     1
8     1                       2           1                     1
9     3                       1           2                     1
10    1                       1           1                     2
11    2                       2           2                     2
12    3                       2           1                     2
13    1

In [6]:
#ordenamos los datasets

muestra = pd.concat([x_muestra, y_muestra], axis=1)
validacion = pd.concat([x_validacion, y_validacion], axis=1)

muestra = muestra.sort_values(by=["target"])
validacion = validacion.sort_values(by=["target"])

print(muestra)

print(validacion)


    Age  Spectacle_prescription  Astigmatic  Tear_production_rate  target
14    1                       2           2                     2       1
10    1                       1           1                     2       2
12    3                       2           1                     2       2
0     3                       2           2                     1       3
1     3                       2           2                     2       3
2     2                       1           1                     1       3
3     3                       2           1                     1       3
4     1                       1           2                     1       3
5     2                       2           2                     1       3
6     3                       1           1                     1       3
7     1                       2           2                     1       3
8     1                       2           1                     1       3
9     3                       1       

In [7]:
#volvemos a dividir en x, y

x_muestra = muestra.drop(["target"], axis=1)
x_validacion = validacion.drop(["target"], axis=1)

y_muestra = muestra["target"]
y_validacion = validacion["target"]


# Aplicamos el algoritmo de Eliminación de candidatos (Candidate Elimination)

In [8]:
#generamos las hipotesis general y especifica

from hipotesis import General, Especifica

import hipotesis


#recordemos que no hablamos de hipotesis general y especifica como unicas hipoesis, sino mas bien nos referimos a un conjunto de hipotesis
hipotesis_generales = [General(x_muestra.columns.tolist())]
hipotesis_especificas = [Especifica(x_muestra.columns.tolist())]

print(corpus.columns)

print(hipotesis_generales[0])
print(hipotesis_especificas[0])

Index(['Age', 'Spectacle_prescription', 'Astigmatic', 'Tear_production_rate'], dtype='object')
<*,*,*,*>
<0,0,0,0>


In [9]:
#como nuestro target son 3 clases, escojamos que es lo que queremos detectar (cuales son los casos positivos)

casos_positivos = [False, False, False] #<- el index-1 de esos booleanos indica que clase consideramos positiva

#Para efectos de esta muestra, vamos a hacer que los casos positivos seran el los primeros y los segundos, que representan a los que si requieren lentes de contacto

casos_positivos[0] = True
casos_positivos[1] = True

In [10]:
from candidate_elimination import Candidtae_Elimination

hipotesis_generales, hipotesis_especifica = Candidtae_Elimination(hipotesis_especificas, hipotesis_generales, x_muestra, y_muestra, casos_positivos)



es esta impresion:  ['Age', 'Spectacle_prescription', 'Astigmatic', 'Tear_production_rate']
es esta impresion:  [[], [], [], []]
[1, 2, 2, 2] 1
Age 1
Spectacle_prescription 2
Astigmatic 2
Tear_production_rate 2
<[1], [2], [2], [2]>
[[1], [2], [2], [2]]
hipotesis que estamos evaluando: <[1], *, *, *>

valor observado: 1	 lista de aceptados: [1]
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
no mochamos nada
hipotesis que estamos evaluando: <[3], *, *, *>

valor observado: 1	 lista de aceptados: [3]
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
mochamos la hipotesis general <[3], *, *, *> porque no cubre al caso observado
hipotesis que estamos evaluando: <[2], *, *, *>

valor observado: 1	 lista de aceptados: [2]
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de

## Ya hemos encontrado el concepto "Usa lentes de contacto". Es decir, la hipotesis acepta valores 1 y 2 en la prediccion, y rechaza el valor 3 en la misma

Ahora es el momento de volver a generar hipotesis, esta vez, con los elementos del dataset que dan como valor de salida 1 o 2

In [11]:
#verificamos la integridad del conjunto de entrenamiento

print(muestra)

    Age  Spectacle_prescription  Astigmatic  Tear_production_rate  target
14    1                       2           2                     2       1
10    1                       1           1                     2       2
12    3                       2           1                     2       2
0     3                       2           2                     1       3
1     3                       2           2                     2       3
2     2                       1           1                     1       3
3     3                       2           1                     1       3
4     1                       1           2                     1       3
5     2                       2           2                     1       3
6     3                       1           1                     1       3
7     1                       2           2                     1       3
8     1                       2           1                     1       3
9     3                       1       

In [12]:
#generamos juntamos el dataframe de muestra nuevamente, para procesarlo para la segunda hipotesis

muestra2 = muestra[muestra['target'] < 3]

x_muestra2 = muestra2.drop(['target'], axis=1)
y_muestra2 = muestra2['target']

casos_positivos2 = [False, True]

print(muestra2)

    Age  Spectacle_prescription  Astigmatic  Tear_production_rate  target
14    1                       2           2                     2       1
10    1                       1           1                     2       2
12    3                       2           1                     2       2


In [13]:
#con esta muestra, entrenamos el nuevo conjunto de hipotesis

hipotesis_generales2 = [General(x_muestra.columns.tolist())]
hipotesis_especificas2 = [Especifica(x_muestra.columns.tolist())]

hipotesis_generales2, hipotesis_especifica2 = Candidtae_Elimination(hipotesis_especificas2, hipotesis_generales2, x_muestra2, y_muestra2, casos_positivos2)

es esta impresion:  ['Age', 'Spectacle_prescription', 'Astigmatic', 'Tear_production_rate']
es esta impresion:  [[], [], [], []]
[1, 2, 2, 2] 1
[[1, 3], [2, 1], [2, 1], [2]]
hipotesis que estamos evaluando: <[1], *, *, *>

valor observado: 1	 lista de aceptados: [1]
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
no mochamos nada
hipotesis que estamos evaluando: <[3], *, *, *>

valor observado: 1	 lista de aceptados: [3]
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
no mochamos nada
hipotesis que estamos evaluando: <*, [2], *, *>

valor observado: 1	 lista de aceptados: []
valor observado: 2	 lista de aceptados: [2]
valor observado: 2	 lista de aceptados: []
valor observado: 2	 lista de aceptados: []
no mochamos nada
hipotesis que estamos evaluando: <*, [1], *, *>

valor observado: 1	 lista de aceptados: []
valor observado: 

In [14]:
# mostramos las hipotesis aprendidas

print(hipotesis_especifica2)

for hipotesis_general2 in hipotesis_generales2:
    print(hipotesis_general2)

<[1, 3], [1, 2], [1], [2]>
<*, *, [1], *>
<*, *, *, [2]>


In [15]:
from random import randint

#con las hipotesis generales, ahora evaluamos la hipotesis aprendida con las metricas de evaluacion

#generamos las predicciones partiendo de la hipotesis general

prediciones_generales:list[list[str]] = []

for hipotesis_general in hipotesis_generales:

    prediccion_general:list[str] = []

    for fila in x_validacion.values.tolist():
        prediccion_general.append(hipotesis.predicciones(fila=fila, hipotesis_general=hipotesis_general, asignacion=[randint(1,2), 3]))
    
    
    prediciones_generales.append(prediccion_general)
    print("la prediccion de la hipotesis general {}: {}".format(hipotesis_general, prediccion_general))


#generamos las predicciones partiendo de la hipotesis especifica

prediccion_especifica:list[str] = []

for fila in x_validacion.values.tolist():
    prediccion_especifica.append(hipotesis.predicciones(fila=fila, hipotesis_especifica=hipotesis_especifica, asignacion=[randint(1,2), 3]))

# print("la prediccion de la hipotesis general:", prediccion_general)
print("la prediccion de la hipotesis especifica:", prediccion_especifica)
print("la salida real: ", y_validacion.values)



len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
este es el contador: 3
len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
este es el contador: 3
len aceptados:  0
len aceptados:  0
len aceptados:  0
len aceptados:  1
la prediccion de la hipotesis general <*, *, *, [2]>: [1, 1, 1, 1, 1, 1, 3, 3, 1]
la prediccion de la hipotesis especifica: [2, 3, 2, 2, 3, 3, 3, 3, 1]
la salida real:  [1 1 1 2 2 2 3 3 3]


# Evaluamos las hipotesis aprendidas

In [16]:
#probamos nuestras hipotesis

#resultados de las hipotesis generales

print("hipotesis generales: \n\n")

for prediccion in prediciones_generales:
    hipotesis.probarHipotesis(prediccion, y_validacion.values.tolist(), [1,2])


#resultados de la hipotesis especifica

print("hipotesis especifica\n\n")
hipotesis.probarHipotesis(prediccion_especifica, y_validacion.values.tolist(), [1,2])

hipotesis generales: 


vp: 6	fp: 1
fn: 0	 vn: 2
accuracy: 0.8888888888888888
precision: 0.8571428571428571
recall: 1.0
f1: 0.923076923076923
hipotesis especifica


vp: 3	fp: 1
fn: 3	 vn: 2
accuracy: 0.5555555555555556
precision: 0.75
recall: 0.5
f1: 0.6


0

## Pero así no nos lo están pidiendo...
## vamos a intentarlo de otra manera

In [17]:
#Digamos que al tratarse del diagnostico de una deficiencia visual, es preferible decir que si debe usar lentes de contacto, sobre que no los debe usar

#Por tal motivo, vamos a asignarle más peso a las hipotesis generales... démosle el 75 por ciento al total de las hipotesis generales
#y dejemosle el 25 a la específica... El 75% se repartirá uniformemente entre tosas las hipotesis generales que salgan

prediccion_final = hipotesis.probarHipotesis2(prediciones_generales, prediccion_especifica)

print(prediccion_final)

[1, 1, 1, 1, 1, 1, 3, 3, 1]


In [18]:
# construimos la matriz de confusión y calculamos las métiricas

hipotesis.probarHipotesis(prediccion_final, y_validacion.values.tolist(), [1,2])

vp: 6	fp: 1
fn: 0	 vn: 2
accuracy: 0.8888888888888888
precision: 0.8571428571428571
recall: 1.0
f1: 0.923076923076923


0